In [38]:
from ortools.sat.python import cp_model
import math

In [61]:

hospitals = [
    {"name": "Hospital A", "location": (34.0522, -118.2437)},  # Los Angeles
    {"name": "Hospital B", "location": (37.7749, -122.4194)},  # San Francisco
    {"name": "Hospital C", "location": (36.1699, -115.1398)},  # Las Vegas
]

beds_per_hospital = [10, 15, 8]  

patients = [
    {"id": 1, "location": (34.0522, -118.2437), "severity": 4},  # Los Angeles
    {"id": 2, "location": (37.7749, -122.4194), "severity": 3},  # San Francisco
    {"id": 3, "location": (36.1699, -115.1398), "severity": 2},  # Las Vegas
    {"id": 4, "location": (34.0522, -118.2437), "severity": 5},  # Los Angeles
]

model = cp_model.CpModel()

x = {}
for i in range(len(hospitals)):
    for k in range(len(patients)):
        x[(i, k)] = model.NewBoolVar(f"x_{i}_{k}")

# Restricciones
# Cada paciente debe ser asignado a un solo hospital
for k in range(len(patients)):
    model.Add(sum(x[(i, k)] for i in range(len(hospitals))) == 1)

# Cada hospital solo puede tener tantos pacientes como camas disponibles
for i in range(len(hospitals)):
    model.Add(sum(x[(i, k)] for k in range(len(patients))) <= beds_per_hospital[i])

# Asignar al paciente al hospital mas cercano en base a distancias euclidianas
distances = []
for i in range(len(hospitals)):
    hospital_loc = hospitals[i]["location"]
    for k in range(len(patients)):
        patient_loc = patients[k]["location"]
        distance = math.sqrt(
            (hospital_loc[0] - patient_loc[0]) ** 2 + (hospital_loc[1] - patient_loc[1]) ** 2
        )
        distances.append(distance * x[(i, k)])


model.Minimize(sum(distances))

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or cp_model.FEASIBLE:
    print("Solucion encontrada")
    assigned_patients = []

    for i in range(len(hospitals)):
        hospital_name = hospitals[i]["name"]
        for k in range(len(patients)):
            if solver.Value(x[(i, k)]) == 1:
                assigned_patients.append(patients[k]["id"])
                patient_id = patients[k]["id"]
                print(f"El paciente {patient_id} esta asignado al hospital {hospital_name}")

    patients_without_assignment = [
        patients[k]["id"] for k in range(len(patients))
        if patients[k]["id"] not in assigned_patients
    ]

    if patients_without_assignment:
        print("Pacientes sin asignacion:")
        for patient in patients_without_assignment:
            print(f"Paciente {patient}")

else:
    print("No se encontro alguna solucion")





¡Solución encontrada!
El paciente 1 está asignado al hospital Hospital A
El paciente 4 está asignado al hospital Hospital A
El paciente 2 está asignado al hospital Hospital B
El paciente 3 está asignado al hospital Hospital C
